In [20]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow, Flow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

import os
from pathlib import Path
import pickle

In [ ]:
"""
from ayv import YouTube

youtube = YouTube.from_secrets(secrets_file='secret_credentials.json', credentials_path='home/user')
youtube = YouTube.from_credentials(credentials_path='/home/user')

youtube.authenticate()
"""

In [73]:
class YouTubeAPIOauthConstants:
    TOKEN_FILE = 'credentials.json'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [78]:
import json
from google.oauth2.credentials import Credentials

In [164]:
class YouTubeAPIAuth:
    __TOKEN_FILE = YouTubeAPIOauthConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIOauthConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIOauthConstants.API_VERSION
    __SCOPES = YouTubeAPIOauthConstants.SCOPES
    
    def __init__(self):
        self.__credentials_path = None
        self.__client_secrets_file = None
        self.__credentials = None
    
    def authenticate_from_client_secrets_file(self, client_secrets_file: str, 
                                              credentials_path: str = ''):
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        if not credentials_path or not os.path.exists(credentials_path):
            self.__credentials_path = self.__get_default_credentials_path()
        else:
            self.__credentials_path = credentials_path
        return self.__from_client_secrets_file()
    
    def authenticate_from_credentials(self, credentials_path: str):
        if not credentials_path:
            raise ValueError('The credentials file path has to be provided.')
        if not isinstance(credentials_path, str):
            raise TypeError('The credentials file should be a string.')
        if not os.path.exists(credentials_path):
            raise ValueError('The credentials file path has to exist!')
        if not Path(credentials_path).is_file():
            raise ValueError('The credentials path must be a file.')
        with open(credentials_path, "r") as credentials:
            self.__credentials = Credentials(**json.load(credentials))
        return self.__from_credentials()
            
    
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
            
    def __get_default_credentials_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        credentials_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return credentials_path
    
    def __from_client_secrets_file(self):
        if os.path.exists(self.__credentials_path):
            with open(self.__credentials_path, "r") as credentials:
                self.__credentials = Credentials(**json.load(credentials))
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
                self.__credentials = flow.run_local_server(port=0)
            with open(self.__credentials_path, "w") as credentials_path:
                credentials = self.__credentials_to_dict(self.__credentials)
                json.dump(credentials, credentials_path)
        youtube_api_client = build(self.__API_SERVICE_NAME, self.__API_VERSION, 
                                   credentials=self.__credentials)
        return youtube_api_client
    
    def __from_credentials(self):
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            with open(self.__credentials_path, "w") as credentials_path:
                credentials = self.__credentials_to_dict(self.__credentials)
                json.dump(credentials, credentials_path)
        youtube_api_client = build(self.__API_SERVICE_NAME, self.__API_VERSION, 
                                   credentials=self.__credentials)
        return youtube_api_client
    
    def __credentials_to_dict(self, credentials: Credentials) -> dict:
        """Convert credentials to a dict for easy work with Flask."""
        return dict(
            token=credentials.token,
            refresh_token=credentials.refresh_token,
            token_uri=credentials.token_uri,
            client_id=credentials.client_id,
            client_secret=credentials.client_secret,
            scopes=credentials.scopes
        )

In [169]:
youtube_api_auth = YouTubeAPIAuth()

In [170]:
client_secrets_file = '/home/lyle/Downloads/client_secret.json'
youtube_api_client = youtube_api_auth.authenticate_from_client_secrets_file(
    client_secrets_file
)

In [172]:
credentials_path = '/home/lyle/credentials.json'
youtube_api_client = youtube_api_auth.authenticate_from_credentials(
    credentials_path
)

In [146]:
class GenerateCredentials:
    __TOKEN_FILE = YouTubeAPIOauthConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIOauthConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIOauthConstants.API_VERSION
    __SCOPES = YouTubeAPIOauthConstants.SCOPES
    
    def __init__(self):
        self.__credentials_path = None
    
    def __get_default_credentials_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        credentials_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return credentials_path

    def __credentials_to_dict(self, credentials: Credentials) -> dict:
        """Convert credentials to a dict for easy work with Flask."""
        return dict(
            token=credentials.token,
            refresh_token=credentials.refresh_token,
            token_uri=credentials.token_uri,
            client_id=credentials.client_id,
            client_secret=credentials.client_secret,
            scopes=credentials.scopes
        )

    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')

    def generate_credentials(self, client_secrets_file: str, credentials_path: str = ''):
        if not credentials_path or not os.path.exists(credentials_path):
            self.__credentials_path = self.__get_default_credentials_path()
        else:
            self.__credentials_path = credentials_path
        flow = Flow.from_client_secrets_file(
            client_secrets_file,
            scopes=["https://www.googleapis.com/auth/youtube.force-ssl"],
            redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        auth_url, _ = flow.authorization_url(prompt='consent')

        print('Please go to this URL: {}'.format(auth_url))
        code = input('Enter the authorization code: ')
        flow.fetch_token(code=code)
        credentials = flow.credentials
        credentials_dict = self.__credentials_to_dict(credentials)
        print(credentials_dict)
        with open(self.__credentials_path, "w") as credentials_path:
            json.dump(credentials_dict, credentials_path)

In [147]:
client_secrets_file = '/home/lyle/Downloads/client_secret.json'
GenerateCredentials().generate_credentials(client_secrets_file)

Please go to this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=690018745938-f53a63e0l2kr7botgudiigblkd87vdlt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=Y0Km7rXIx5I2UEVxGZT1fxkaqxd6fz&prompt=consent&access_type=offline
Enter the authorization code: 4/1AWtgzh5wh8tX9SX76SELfKWNlo9kY5LHNxgRX3FuB4Gjg9tsVjNGno_6HE4
{'token': 'ya29.a0AVvZVspqG3kdBy34mQjrz5t6gOorAzN6T9_NeV1ZQYFKn2ZukpdgOtZVEBE0YgWBMpGixMOQzWU-jCsuiIpp0sA7LUjJJJ_DRLJUhFjmGfynQOry_HDlJPGXDVkFSKWr4LxSe5WVdy6bwVbFxZBRfGK-MCdZaCgYKAW4SARESFQGbdwaIvi_gfgFluxI0iG2TIELJlA0163', 'refresh_token': '1//03Xyi0enZMfiXCgYIARAAGAMSNwF-L9IrpccCIpF-yXmx3Y-97nRPm6TUY4xNyVroBsToT9_J3dcLZmqKkcCb6y4VgiqCgK10DzU', 'token_uri': 'https://oauth2.googleapis.com/token', 'client_id': '690018745938-f53a63e0l2kr7botgudiigblkd87vdlt.apps.googleusercontent.com', 'client_secret': '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', 'scopes'

In [163]:
credentials_path = '/home/lyle/credentials.json'
youtube_api_client = youtube_api_auth.authenticate_from_credentials(
    credentials_path
)

{'token': 'ya29.a0AVvZVsqNlYdrpJE74el96Y1mNhQ2MgkECUFHwiHJQoTDQkT_k7rp4SjjI1PPtVstOWGNAKdgL-chDun1kZS5_OO4x2nriPHF25uii5SwZZTJspCUfRSZfKpyRVIj3Oxukt-MSglETtWGDNU19ZDMbPqE7fr8aCgYKAUgSARESFQGbdwaIUZKDEfimtjE7JhTEKBxs_w0163', 'refresh_token': '1//03OW6yrqnVofACgYIARAAGAMSNwF-L9IrmNTvnFqLDRE6AU6YAyTBg8Lv-750abiN7o_Xwo3TFdm2GjdaVQYJ2QUXozJV6HFxctw', 'token_uri': 'https://oauth2.googleapis.com/token', 'client_id': '690018745938-f53a63e0l2kr7botgudiigblkd87vdlt.apps.googleusercontent.com', 'client_secret': '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', 'scopes': ['https://www.googleapis.com/auth/youtube.force-ssl']}


In [173]:
youtube_api_client.videos().list(
        part="snippet,contentDetails,statistics",
        id='hG8KEfG5lwg'
    ).execute()

{'kind': 'youtube#videoListResponse',
 'etag': 'fF20te3_MAloBOsF4H-A5j11yg0',
 'items': [{'kind': 'youtube#video',
   'etag': 'W-f5oMGyFiedz3BsVTXcSYo5wHM',
   'id': 'hG8KEfG5lwg',
   'snippet': {'publishedAt': '2023-02-16T23:00:52Z',
    'channelId': 'UCsgPO6cNV0wBG-Og3bUZoFA',
    'title': 'Gulag - The Story | Part 2: Propagation - 1934 - 1945 | Free Documentary History',
    'description': "Gulag - The Story - Part 2: Propagation - 1934 - 1945 | History Documentary\n\nWatch 'Gulag  - The Story: Part 3' here: https://youtu.be/dBjcT0QxSS0\n\nA major political, historical, human and economic fact of the 20th century, the Gulag, the extremely punitive Soviet concentration camp system, remains largely unknown.\nThe history of the Gulag is long, complex and in many ways out of the ordinary. From the Revolution of 1917 to Gorbachev, touching on the civil war, the Great Terror, World War II, the Cold War and the death of Stalin, this series describes the workings of the Gulag.\nHow and why 